### CMF - Matrix Factorization

Baseline

In [15]:
# Imports

#!pip install cmfrec
from cmfrec import CMF_implicit # Collective Matrix Factorization Lib
from sklearn.model_selection import train_test_split

import pandas as pd
from src.evaluator import Evaluator

In [27]:
# Set up data generation class

class TrainTestGenerator:
    def __init__(self: str):
        pass

    def prepare_data(self):
        df_user_interactions = pd.read_csv("data/rated_activities.csv")        
        return df_user_interactions

    def forward_chaining(self): # no timestamp in data
        data = self.prepare_data()

        values = [0.3, 0.2, 0.1]
        
        for test_sample_size in values:
            train_df, test_df = train_test_split(data, test_size=test_sample_size, random_state=42) # 42 for reproducability
            
            #train = data[data["user_id"].dt.year < test_year]
            #test = data[data["timestamp"].dt.year == test_year]

            yield test_sample_size, train_df, test_df
            
data_generator = TrainTestGenerator()
interaction_df = data_generator.prepare_data()
interaction_df.head()

,item_id,user_id,rating_id
0,206,1,9
1,1014,1,3
2,231,1,7
3,725,1,8
4,1148,1,2


In [28]:
# Set up model class
class CMF_recommender:
    def __init__(self, k=50):
        self.model = CMF_implicit(
            k=k,
            random_state=1,
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data["weight"] = 1  # Binary adjacency matrix (no weights)
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

In [29]:
evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/cmfrec/__init__.py:138: UserWarning: Attempting to use more than 1 thread, but package was built without multi-threading support - see the project's GitHub page for more information.
  warnings.warn(msg_omp)


AssertionError: 